# Практика

Скоринг учащихся (следующий семестр на основе текущего, количество 2/3/4/5)

## Загрузка библиотек

In [1]:
!pip install catboost
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import Pool, CatBoost, CatBoostRegressor, cv

## Предобработка данных

In [3]:
pre_data = pd.read_excel('Успеваемость_01.xlsx')

In [4]:
pre_data.to_csv('/content/new_data.csv', index=False)

In [5]:
df_new = pd.read_csv('new_data.csv')

In [6]:
df_new.head(1)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Инженерная компьютерная графика,Хорошо,Хорошо


In [7]:
df_new = df_new.replace({'I полугодие': 1, 'II полугодие': 2})

In [8]:
years = df_new['Учебный год'].unique()
years.sort()
print(f'Присутствуют записи за: {years} года')

Присутствуют записи за: ['2017 - 2018' '2018 - 2019' '2019 - 2020' '2020 - 2021' '2021 - 2022'
 '2022 - 2023' '2023 - 2024' '2024 - 2025' '2025 - 2026' '2026 - 2027'
 '2027 - 2028'] года


In [9]:
print(f'Всего записей: {len(df_new)}')

Всего записей: 376007


#### Удаление данных

Удаление учебных годов (практически) без оценок

In [10]:
years_to_exclude = ['2023 - 2024', '2024 - 2025', '2025 - 2026', '2026 - 2027', '2027 - 2028']

df_new = df_new[~df_new['Учебный год'].isin(years_to_exclude)].copy()

In [11]:
df_new = df_new[~df_new['Учебная группа'].str.contains('22')].copy()

Удаление предметов без оценки

In [12]:
df_new = df_new.dropna(subset=['Оценка (без пересдач)', 'Оценка (успеваемость)'], how='all')

#### Заполнение пропусков в промежуточной аттестации

при условии, что есть итоговая оценка

In [13]:
df_new['Оценка (без пересдач)'] = df_new.apply(
    lambda row: 2 if pd.isna(row['Оценка (без пересдач)']) and not pd.isna(row['Оценка (успеваемость)']) else row['Оценка (без пересдач)'],
    axis=1
)

#### Добавление столбца с семестром

In [14]:
def calculate_semester(row):
    group_year = int(row['Учебная группа'].split('-')[1])
    start_year = int(row['Учебный год'].split(' - ')[0])
    course = (start_year % 100) - group_year + 1
    semester = course * 2 - 1 if row['Полугодие'] == 1 else course * 2
    return semester

df_new['Семестр'] = df_new.apply(calculate_semester, axis=1)

In [15]:
df_new['Программа'] = df_new['Учебная группа'].str.split('-').str[0]

In [16]:
df_new.head(1)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,1,Инженерная компьютерная графика,Хорошо,Хорошо,1,БИВТ


In [17]:
disciplines = len(df_new['Дисциплина'].unique())
print(f'Количество уникальных дисциплин: {disciplines}')

Количество уникальных дисциплин: 658


In [18]:
programs = len(df_new['Программа'].unique())
print(f'Количество уникальных программ: {programs}')

Количество уникальных программ: 19


In [19]:
df = df_new
df = df.drop(columns=['Номер ЛД', 'Учебная группа', 'Уровень подготовки', 'Учебный год', 'Полугодие', 'Специальность/направление'])

In [20]:
grades = df['Оценка (успеваемость)'].unique()
print(grades)

['Хорошо' 'Удовлетворительно' 'зачтено' 'Отлично' nan
 'Неудовлетворительно' 'Неявка' 'не зачтено' 'Неявка по ув.причине'
 'Не допущен']


In [21]:
df.head(1)

,hash,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,Инженерная компьютерная графика,Хорошо,Хорошо,1,БИВТ


#### Кодирование оценок

In [22]:
df.replace({'зачтено': 5, 'Отлично': 5,
            'Хорошо': 4,
            'Удовлетворительно': 3,
            'Неудовлетворительно': 2, 'Неявка': 2, 'не зачтено': 2, 'Не допущен': 2,
            'Неявка по ув.причине': 0
            },
           inplace=True)

Так как по ТЗ все оставшиеся null у студента это 2, то заменим их:

In [23]:
df.fillna(2, inplace=True)

In [24]:
df.head(2)

,hash,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,Инженерная компьютерная графика,4,4.0,1,БИВТ
1,67a80fffd8d0294a596eda117d7e393c,Иностранный язык,3,3.0,1,БИВТ


In [25]:
df.groupby(['Семестр'], sort=False).size().reset_index(name='cnt')

,Семестр,cnt
0,1,23145
1,2,27380
2,3,24769
3,4,32672
4,5,15204
5,6,15545
6,7,2458
7,8,3145
8,9,1012
9,10,1162


#### Создание столбцов для каждого направления

In [26]:
df_programs = pd.get_dummies(df, columns=['Программа'], prefix='', prefix_sep='')
df_programs = df_programs.drop(columns=['Дисциплина', 'Оценка (без пересдач)', 'Оценка (успеваемость)'])
df_programs = df_programs.astype({col: int for col in df_programs.columns[1:]})
df_programs = df_programs.drop_duplicates()

In [27]:
df_programs.head()

,hash,Семестр,ББИ,БИВТ,БИСТ,БЛГ,БМН,БМТ,БМТМ,БНМ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,67a80fffd8d0294a596eda117d7e393c,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,67a80fffd8d0294a596eda117d7e393c,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,67a80fffd8d0294a596eda117d7e393c,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,67a80fffd8d0294a596eda117d7e393c,4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,67a80fffd8d0294a596eda117d7e393c,5,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Создание столбцов для каждой дисциплины

Сопоставление промежуточной оценки соответствующей дисциплине

In [28]:
pivot_df = df.pivot_table(index=['hash', 'Семестр'],
                          columns='Дисциплина',
                          values='Оценка (без пересдач)',
                          aggfunc='first'
                          ).reset_index()

pivot_df.columns.name = None
pivot_df.columns = [str(col) for col in pivot_df.columns]

Заполнение отсутствующих предметов нулями

In [29]:
pivot_df.fillna(0, inplace=True)

In [30]:
pivot_df = pivot_df.astype({col: int for col in pivot_df.columns[1:]})

In [31]:
pivot_df.head(2)

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Подсчёт количества двоек в каждом семестре

In [32]:
df = pivot_df
df.head(7)

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,000006af6e40c8234a5af27896b7bba5,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,00083ac3c8aecc3a1cf66029173e56fa,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
grades_columns = df.drop(['hash', 'Семестр'], axis=1)

In [34]:
df['count_2'] = (df == 2).sum(axis=1)
df.head(7)

<ipython-input-34-98367a2bc522>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['count_2'] = (df == 2).sum(axis=1)


,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,count_2
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
5,000006af6e40c8234a5af27896b7bba5,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
6,00083ac3c8aecc3a1cf66029173e56fa,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Смещение количества двоек наверх на одну строку для каждого студента
df['next_sem_debts'] = df['count_2'].shift(-1).copy()
# Если встретился следующий студент - последнему известному семестру текущего
# студента соответсвует данное количество долгов без смещения
df['next_sem_debts'] = np.where(df['hash'] != df['hash'].shift(-1), df['count_2'], df['next_sem_debts'])
df['next_sem_debts'].fillna(df['count_2'], inplace=True)
df = df.drop('count_2', axis=1)

<ipython-input-35-6ae964e7966c>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['next_sem_debts'] = df['count_2'].shift(-1).copy()


In [36]:
df['next_sem_debts'] = df['next_sem_debts'].astype(int)

In [37]:
df

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,next_sem_debts
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18094,ffed4fe2ad8cbff97e0b3d21da23008d,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
18095,ffed4fe2ad8cbff97e0b3d21da23008d,5,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,7
18096,ffed4fe2ad8cbff97e0b3d21da23008d,6,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
18097,ffed4fe2ad8cbff97e0b3d21da23008d,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


#### Подсчёт количества каждого вида оценок

In [38]:
subjects = df.apply(pd.Series.value_counts, axis=1)[[2, 3, 4, 5]].fillna(0)
subjects = subjects.astype(int)

#subjects['total'] = subjects[[2, 3, 4, 5]].sum(axis=1)
#subjects['total'] = pd.to_numeric(subjects['total'], errors='coerce').fillna(0)
subjects.head()

,2,3,4,5
0,2,2,1,3
1,4,3,2,2
2,4,2,2,2
3,4,3,1,2
4,7,1,1,1


In [39]:
df.head()

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,next_sem_debts
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


#### Смещение столбцов

In [40]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
print(cols)

['next_sem_debts', 'hash', 'Семестр', 'BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений', 'BIM-технологии при проектировании горнодобывающих комплексов', 'CAD системы в горном производстве', 'CAD системы в подземном строительстве', 'Cертификация в горном деле', 'HR-системы организационного управления предприятием', 'Linux', 'Python для извлечения и обработки данных', 'Автоматизация горных машин и установок', 'Автоматизация письменного перевода и управление терминологией', 'Автоматизация технологических процессов', 'Автоматизированное проектирование машин', 'Автоматизированный электропривод машин и установок', 'Автомобили и тракторы', 'Академический английский, часть 1', 'Академический английский, часть 2', 'Академический английский. Часть 1', 'Алгоритмы анализа данных', 'Алгоритмы дискретной математики', 'Алгоритмы и структуры данных', 'Анализ данных и аналитика в принятии решений', 'Анализ данных на практике', 'Анализ и моделирование бизнес-процессов'

In [41]:
df = df[cols]

In [42]:
lessons = df.columns.to_list()[2:]
print(lessons)

['Семестр', 'BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений', 'BIM-технологии при проектировании горнодобывающих комплексов', 'CAD системы в горном производстве', 'CAD системы в подземном строительстве', 'Cертификация в горном деле', 'HR-системы организационного управления предприятием', 'Linux', 'Python для извлечения и обработки данных', 'Автоматизация горных машин и установок', 'Автоматизация письменного перевода и управление терминологией', 'Автоматизация технологических процессов', 'Автоматизированное проектирование машин', 'Автоматизированный электропривод машин и установок', 'Автомобили и тракторы', 'Академический английский, часть 1', 'Академический английский, часть 2', 'Академический английский. Часть 1', 'Алгоритмы анализа данных', 'Алгоритмы дискретной математики', 'Алгоритмы и структуры данных', 'Анализ данных и аналитика в принятии решений', 'Анализ данных на практике', 'Анализ и моделирование бизнес-процессов', 'Анализ точности маркшей

In [43]:
df.head(3)

,next_sem_debts,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,4,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df = df.astype({col: int for col in df.columns[2:]})

In [45]:
df.head(3)

,next_sem_debts,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,4,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Объединение таблицы оценок с таблицей направлений

In [46]:
df_programs.head(1)

,hash,Семестр,ББИ,БИВТ,БИСТ,БЛГ,БМН,БМТ,БМТМ,БНМ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,67a80fffd8d0294a596eda117d7e393c,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
df_merged = pd.merge(df, df_programs, on=['hash', 'Семестр'], how='inner')

In [48]:
df_merged

,next_sem_debts,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,4,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18099,7,ffed4fe2ad8cbff97e0b3d21da23008d,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18100,7,ffed4fe2ad8cbff97e0b3d21da23008d,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18101,8,ffed4fe2ad8cbff97e0b3d21da23008d,6,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
18102,3,ffed4fe2ad8cbff97e0b3d21da23008d,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Удаление столбцов с дисциплинами

т.к. учитывается только количество оценок каждого вида

In [49]:
column_list = df_merged.columns.to_list()
column_list = column_list[:3] + column_list[-19:]
print(column_list)

['next_sem_debts', 'hash', 'Семестр', 'ББИ', 'БИВТ', 'БИСТ', 'БЛГ', 'БМН', 'БМТ', 'БМТМ', 'БНМ', 'БНМТ', 'БПИ', 'БПМ', 'БТМО', 'БФЗ', 'БЭК', 'БЭН', 'БЭЭ', 'СГД', 'СНТС', 'СФП']


In [ ]:
df_merged_cnt = df_merged[column_list]
df_merged_cnt = subjects.join(df_merged_cnt, how='inner')

In [54]:
df = df_merged_cnt

In [55]:
df.head(3)

,2,3,4,5,next_sem_debts,hash,Семестр,ББИ,БИВТ,БИСТ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,2,2,1,3,4,000006af6e40c8234a5af27896b7bba5,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,3,2,2,4,000006af6e40c8234a5af27896b7bba5,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,2,2,2,4,000006af6e40c8234a5af27896b7bba5,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = df.drop('hash', axis=1)

## Обучение модели

In [59]:
results = df['next_sem_debts'].to_numpy()
print(f'Минимальное количество двоек: {results.min()}')
print(f'Максимальное количество двоек: {results.max()}')
print(f'Среднее количество двоек: {results.mean():.3f}')

Минимальное количество двоек: 0
Максимальное количество двоек: 15
Среднее количество двоек: 2.225


### Обучение с учётом направления

#### Разделение на трейн-тест

In [58]:
X = df.drop('next_sem_debts', axis=1)
X = X.round().astype(int)
y = df['next_sem_debts']
y = y.astype(int)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [61]:
X_train

,2,3,4,5,Семестр,ББИ,БИВТ,БИСТ,БЛГ,БМН,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
3905,3,0,1,6,6,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
14804,4,0,1,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9185,0,0,3,7,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1675,1,0,1,7,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6932,8,2,1,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,0,0,1,7,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11964,0,0,0,8,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5390,3,2,1,4,6,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
860,1,0,1,6,6,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [62]:
y_train

3905     2
14804    4
9185     1
1675     0
6932     5
        ..
11284    0
11964    0
5390     2
860      1
15795    0
Name: next_sem_debts, Length: 13574, dtype: int64

In [63]:
train_pool = Pool(X_train,
                  label=y_train)

test_pool = Pool(X_test,
                 label=y_test)

### Подбор гиперпараметров

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_seed': 42,
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'loss_function': 'RMSE',
        'eval_metric': 'R2',
    }
    model = CatBoostRegressor(**params, verbose=False)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print('Best trial:')
best_trial = study.best_trial
print(f'  Value: {best_trial.value:.4f}')
print('  Params: ')
for key, value in best_trial.params.items():
    print(f'    {key}: {value}')

[I 2024-07-21 06:42:43,122] A new study created in memory with name: no-name-82fb5442-f9c1-4b7f-8694-f944814331ea
[I 2024-07-21 06:42:43,471] Trial 0 finished with value: 2.064637923682137 and parameters: {'iterations': 302, 'depth': 5, 'learning_rate': 0.14637018726028483, 'l2_leaf_reg': 1.8739122418095797}. Best is trial 0 with value: 2.064637923682137.
[I 2024-07-21 06:42:44,112] Trial 1 finished with value: 2.072659567486985 and parameters: {'iterations': 116, 'depth': 8, 'learning_rate': 0.04099082738299252, 'l2_leaf_reg': 5.761207787616964}. Best is trial 0 with value: 2.064637923682137.
[I 2024-07-21 06:42:44,741] Trial 2 finished with value: 2.0453275160583324 and parameters: {'iterations': 271, 'depth': 9, 'learning_rate': 0.2369636501732365, 'l2_leaf_reg': 7.473144100943668}. Best is trial 2 with value: 2.0453275160583324.
[I 2024-07-21 06:42:45,270] Trial 3 finished with value: 2.0519746747370706 and parameters: {'iterations': 871, 'depth': 9, 'learning_rate': 0.238994758312

Best trial:
  Value: 2.0360
  Params: 
    iterations: 348
    depth: 10
    learning_rate: 0.0691967279589776
    l2_leaf_reg: 9.678946897271532


In [64]:
model = CatBoostRegressor(iterations=300, custom_metric='R2', learning_rate=0.07, depth=10, l2_leaf_reg=9.68)
model.fit(train_pool, eval_set=[test_pool], verbose=200)

0:	learn: 2.4647439	test: 2.4327967	best: 2.4327967 (0)	total: 55.2ms	remaining: 16.5s
200:	learn: 1.9212186	test: 2.0374482	best: 2.0372442 (196)	total: 1.22s	remaining: 600ms
299:	learn: 1.8586629	test: 2.0396505	best: 2.0367506 (248)	total: 1.79s	remaining: 0us

bestTest = 2.036750599
bestIteration = 248

Shrink model to first 249 iterations.


In [65]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'RMSE: {rmse:.4f}')
print(f'R2: {r2:.4f}')

RMSE: 2.0368
R2: 0.3210
